In [1]:
#Per impostare il locale italiano
# 1 Decommentare l'ultima riga ed eseguire la cella
# 2 Selezionare la codifica it_IT.UTF-8 '282'
# 3 Riavviare il runtime ed eseguire questa cella con l'ultima riga commentata

!export LC_ALL="it_IT.UTF-8"
!export LC_CTYPE="it_IT.UTF-8"
#!sudo dpkg-reconfigure locales

In [12]:
import locale
from dask.diagnostics import ProgressBar  
locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')

'it_IT.UTF-8'

In [3]:
!pip install aiohttp
!pip install requests
!pip install dask[dataframe] --upgrade



     |████████████████████████████████| 1.1 MB 6.0 MB/s 
     |████████████████████████████████| 144 kB 6.8 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 271 kB 6.7 MB/s 
     |████████████████████████████████| 133 kB 27.3 MB/s 


In [4]:
import dask.dataframe as dd
import math

In [5]:
class BaseDfBench(object):
    def __init__(self, df):
      self.df = df

    def load_dataset(self, path, format, conn=None, **kwargs):
        """
        Load the provided dataframe
        """
        if format == "csv":
            self.df = self.read_csv(path, **kwargs)
        elif format == "json":
            self.df = self.read_json(path, **kwargs)
        elif format == "xml":
            self.df = self.read_xml(path, **kwargs)
        elif format == "excel":
            self.df = self.read_excel(path, **kwargs)
        elif format == "parquet":
            self.df = self.read_parquet(path, **kwargs)
        elif format == "sql": 
            self.df = self.read_sql(path, conn, **kwargs)            
        return self.df        
        
    def read_sql(self, query, conn, **kwargs):
        """
        Given a connection and a query
        creates a dataframe from the query output
        """
        self.df = dd.read_sql(query, conn)
        return self.df
    def read_json(self, path, **kwargs):
        """
        Read a json file
        """
        self.df = dd.read_json(path, **kwargs)
        return self.df
    
    def read_csv(self, path, **kwargs):
        """
        Read a csv file
        """
        self.df = dd.read_csv(path, **kwargs)
        return self.df
        
    def read_xml(self, path, **kwargs):
        """
        Read a xml file
        """
        self.df = dd.read_xml(path, **kwargs)
        return self.df
        
    def read_excel(self, path, **kwargs):
        """
        Read an excel file
        """
        self.df = dd.read_excel(path, **kwargs)
        return self.df
        
    def read_parquet(self, path, **kwargs):
        """
        Read a parquet file
        """
        self.df = dd.read_parquet(path, **kwargs)
        return self.df
    def sort(self, columns, ascending=True):
        """
        Sort the dataframe by the provided columns
        Columns is a list of column names
        """
        self.df = self.df.sort_values(columns, ascending=ascending)
        return self.df

    def get_columns(self):
        """
        Return the name of the columns in the dataframe
        """
        return list(self.df.columns.values)

    def is_unique(self, column):
        """
        Check the uniqueness of all values contained in the provided column_name
        """
        return self.df[column].is_unique

    def delete_columns(self, columns):
        """
        Delete the specified columns
        Columns is a list of column names
        """
        self.df = self.df.drop(columns=columns)
        return self.df

    def rename_columns(self, columns):
        """
        Rename the provided columns using the provided names
        Columns is a dictionary: {"column_name": "new_name"}
        """
        self.df = self.df.rename(columns=columns)
        return self.df

    def merge_columns(self, columns, separator, name):
        """
        Create a new column with the provided name combining the two provided columns using the provided separator
        Columns is a list of two column names; separator and name are strings
        """
        self.df[name] = self.df[columns[0]].astype(str) + separator + self.df[columns[1]].astype(str)
        return self.df

    def fill_nan(self, value):
        """
        Fill nan values in the dataframe with the provided value
        """
        self.df = self.df.fillna(value)
        return self.df
        
    def one_hot_encoding(self, columns):
        """
        Performs one-hot encoding of the provided columns
        Columns is a list of column names
        """
        dummies = dd.get_dummies(self.df[columns])
        self.df = dd.concat([self.df.drop(columns=columns), dummies], axis=1)
        return self.df

    def locate_null_values(self, column):
        """
        Returns the rows of the dataframe which contains
        null value in the provided column.
        """
        return self.df[self.df[column].isna()]
    def search_by_pattern(self, column, pattern):
        """
        Returns the rows of the dataframe which
        match with the provided pattern
        on the provided column.
        Pattern could be a regular expression.
        """
        return self.df[self.df[column].str.contains(re.compile(pattern))]
        
    def locate_outliers(self, column, lower_quantile=0.1, upper_quantile=0.99):
        """
        Returns the rows of the dataframe that have values
        in the provided column lower or higher than the values
        of the lower/upper quantile.
        """
        q_low = self.df[column].quantile(lower_quantile)
        q_hi  = self.df[column].quantile(upper_quantile)
        return self.df[(self.df[column] < q_low) | (self.df[column] > q_hi)]
        
    def get_columns_types(self):
        """
        Returns a dictionary with column types
        """
        return self.df.dtypes.apply(lambda x: x.name).to_dict()
        
    def cast_columns_types(self, converterList: list, str_date_time_format = '%d %B %Y'):
        """
        Cast the data types of the provided columns 
        to the provided new data types.
        dtypes is a dictionary that provide for each
        column to cast the new data type.
        """
        columnsDate = []
        for item in converterList:
            if(item['correct_dtype'] == float):
              #self.df[item['col']] = 
              #self.df[item['col']].str.replace('.', '').str.replace(',', '.').str.replace('\U00002013', '-').str.strip('€/smc').astype(float)
              self.df[item['col']] = self.df[item['col']].str.replace('.', '').str.replace(',', '.').str.extract('([+-]?[0-9]+\.[0-9]+)', expand=False).astype(float)
            elif(item['correct_dtype'] == 'datetime'):
              columnsDate.append(item['col'])
        
        self.change_date_time_format(columnsDate, str_date_time_format = '%d %B %Y')
        return self.df
        
        
    def get_stats(self):
        """
        Returns dataframe statistics.
        Only for numeric columns.
        Min value, max value, average value, standard deviation, and standard quantiles.
        """
        return self.df.describe()
        
    
    def assign_custom_types(self, mismatched_types, correct_types):

      for mismatch in mismatched_types:
        mismatch['correct_dtype'] = correct_types[mismatch['col']]
      
      return mismatched_types


        
    def find_mismatched_dtypes(self):
      """
      L'implementazione originaria non si adattava alla nostra libreria.
      Dask interpreta i mismatch nella colonne come object, per cui facendo un rapido controllo 
      su questo tipo di dato, otteniamo le colonne che dobbiamo andare a sistemare.
      Il risultato è un Set con due liste che riportano rispettivamente i nomi delle colonne con type=object
      e la seconda lista i rispettivi indici.
      """
      current_dtypes = self.get_columns_types()

      out = []
      for k in current_dtypes.keys():
          if current_dtypes[k] == 'object':
            out.append({'col': k, 'current_dtype': current_dtypes[k], 'correct_dtype': ''})
      return out
        
    def check_allowed_char(self, column, pattern):
        """
        Return true if all the values of the provided column
        follow the provided pattern.
        For example, if the pattern [a-z] is provided the string
        'ciao' will return true, the string 'ciao123' will return false.
        """
        return self.df[column].str.contains(re.compile(pattern)).all()
        
    def drop_duplicates(self):
        """
        Drop duplicate rows.
        """
        self.df = self.df.drop_duplicates()
        return self.df
        
    def drop_by_pattern(self, column, pattern):
        """
        Delete the rows where the provided pattern
        occurs in the provided column.
        """
        matching_rows = self.search_by_pattern(column, pattern)
        self.df = self.df.drop(matching_rows.index)
        return self.df
        
    def change_date_time_format(self, columns: list, str_date_time_format):
        """
        Change the date/time format of the provided column
        according to the provided formatting string.
        column datatype must be datetime
        An example of str_date_time_format is '%m/%d/%Y'
        """
        for col in columns:
          self.df[col] = dd.to_datetime(self.df[col], format=str_date_time_format, dayfirst=True, errors='coerce')
        return self.df
        
    def set_header_case(self, case):
        """
        Put dataframe headers in the provided case
        Supported cases: "lower", "upper", "title", "capitalize", "swapcase"
        (see definitions in pandas documentation)
        """
        if mode == "lower":
            self.df.columns = map(str.lower, self.df.columns)
        elif mode == "upper":
            self.df.columns = map(str.upper, self.df.columns)
        elif mode == "title":
            self.df.columns = map(str.title, self.df.columns)
        elif mode == "capitalize":
            self.df.columns = map(str.capitalize, self.df.columns)
        elif mode == "swapcase":
            self.df.columns = map(str.swapcase, self.df.columns)
        return self.df

    def set_content_case(self, columns, case):
        """
        Put dataframe content in the provided case
        Supported cases: "lower", "upper", "title", "capitalize", "swapcase"
        (see definitions in pandas documentation)
        Columns is a list of two column names; empty list for the whole dataframe
        """
        if len(columns) == 0:
            columns = list(self.df.columns.values)
        for column in columns:
            if mode == "lower":
                self.df[column] = self.df[column].str.lower()
            elif mode == "upper":
                self.df[column] = self.df[column].str.upper()
            elif mode == "title":
                self.df[column] = self.df[column].str.title()
            elif mode == "capitalize":
                self.df[column] = self.df[column].str.capitalize()
            elif mode == "swapcase":
                self.df[column] = self.df[column].str.swapcase()
        return self.df

    def duplicate_columns(self, columns):
        """
        Duplicate the provided columns (add to the dataframe with "_duplicate" suffix)
        Columns is a list of column names
        """
        for column in columns:
            self.df[column + "_duplicate"] = self.df[column]
        return self.df

    def pivot(self, index, columns, values, aggfunc):
        """
        Define the lists of columns to be used as index, columns and values respectively,
        and the dictionary to aggregate ("sum", "mean", "count") the values for each column: {"col1": "sum"}
        (see pivot_table in pandas documentation)
        """
        self.df = dd.pivot_table(self.df, index=index, values=values, columns=columns, aggfunc=aggfunc).reset_index()
        return self.df

    def unpivot(self, columns, var_name, val_name):
        """
        Define the list of columns to be used as values for the variable column,
        the name for variable columns and the one for value column_name
        """
        self.df = dd.melt(self.df, id_vars=list(set(list(self.df.columns.values)) - set(columns)), value_vars=columns, var_name=var_name, value_name=val_name)
        return self.df

    def delete_empty_rows(self, columns):
        """
        Delete the rows with null values for all provided Columns
        Columns is a list of column names
        """
        self.df = self.df.dropna(subset = columns, inplace=True)
        return self.df

    def split(self, column, sep, splits, col_names):
        """
        Split the provided column into splits + 1 columns named after col_names
        using the provided sep string as separator
        Col_names is a list of column names
        """
        self.df[col_names] = self.df[column].str.split(sep, splits, expand=True)
        return self.df

    def strip(self, columns, chars):
        """
        Remove the characters appearing in chars at the beginning/end of the provided columns
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column].str.strip(chars)
        return self.df

    def remove_diacritics(self, columns):
        """
        Remove diacritics from the provided columns
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        return self.df
        
    def set_index(self, column):
        """
        Set the provided column as index
        """
        self.df = self.df.set_index(column)
        return self.df
        
        
    def change_num_format(self, formats):
        """
        Round one ore more columns to a variable number of decimal places.
        formats is a dictionary with the column names as key and the number of decimal places as value.
        """
        self.df = self.df.round(formats)
        return self.df
        
        
    def calc_column(self, col_name, f):
        """
        Calculate the new column col_name by applying
        the function f
        """
        self.df[col_name] = self.df.apply(f, axis=1)
        return self.df
        
    def join(self, other, left_on=None, right_on=None, how='inner', **kwargs):
        """
        Joins current dataframe (left) with a new one (right).
        left_on/right_on are the keys on which perform the equijoin
        how is the type of join
        **kwargs: additional parameters
        
        The result is stored in the current dataframe.
        """
        self.df = self.df.merge(other, left_on=left_on, right_on=right_on, how=how, **kwargs)
        return self.df
        
    def groupby(self, columns, f):
        """
        Aggregate the dataframe by the provided columns
        then applied the function f on every group
        """
        return self.df.groupby(columns).agg(f)
        
    
    def categorical_encoding(self, columns):
        """
        Convert the categorical values in these columns into numerical values
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column].astype('category')
            self.df[column] = self.df[column].cat.codes
        return self.df

    def sample_rows(self, frac, num):
        """
        Return a sample of the rows of the dataframe
        Frac is a boolean:
        - if true, num is the percentage of rows to be returned
        - if false, num is the exact number of rows to be returned
        """
        if frac:
            return self.df.sample(frac=num/100)
        else:
            return self.df.sample(n=num)

    def append(self, other, ignore_index):
        """
        Append the rows of another dataframe (other) at the end of the provided dataframe
        All columns are kept, eventually filled by nan
        Ignore index is a boolean: if true, reset row indices
        """
        self.df = self.df.append(other, ignore_index=ignore_index)
        return self.df

    def replace(self, columns, to_replace, value, regex):
        """
        Replace all occurrencies of to_replace (numeric, string, regex, list, dict) in the provided columns using the provided value
        Regex is a boolean: if true, to_replace is interpreted as a regex
        Columns is a list of column names
        """
        self.df[columns] = self.df[columns].replace(to_replace=to_replace, value=value, regex=regex)
        return self.df

    def edit(self, columns, func):
        """
        Edit the values of the cells in the provided columns using the provided expression
        Columns is a list of column names
        """
        self.df[columns] = self.df[columns].apply(func)
        return self.df

    def set_value(self, index, column, value):
        """
        Set the cell identified by index and column to the provided value
        """
        self.df.at[index, column] = value
        return self.df

    def min_max_scaling(self, columns):
        """
        Independently scale the values in each provided column in the range (0, 1)
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column] - self.df[column].min()
            self.df[column] = self.df[column] / self.df[column].max()
            self.df[column] = self.df[column] * (max - min) + min
        return self.df

    def round(self, columns, n):
        """
        Round the values in columns using n decimal places
        Columns is a list of column names
        """
        self.df[columns] = self.df[columns].round(n)
        return self.df
        
    def get_duplicate_columns(self):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """
        cols = self.df.columns.values
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
    
    def to_csv(self, path, **kwargs):
        """
        Export the dataframe in a csv file.
        """
        self.df.to_csv(path, **kwargs)
        pass
        
    def query(self, query):
        """
        Queries the dataframe and returns the corresponding
        result set.
        :param query: a string with the query conditions, e.g. "col1 > 1 & col2 < 10"
        :return: subset of the dataframe that correspond to the selection conditions
        """
        return self.df.query(query)

In [6]:
from pathlib import Path
import pandas as pd


path = 'https://dbgroup.ing.unimore.it/invoices/data.zip'
!wget -nc 'https://dbgroup.ing.unimore.it/invoices/data.zip'
!unzip '/content/data.zip'


#dtype={'billing_frequency': 'string','gas_offer': 'float64'}

--2022-01-29 09:06:47--  https://dbgroup.ing.unimore.it/invoices/data.zip
Resolving dbgroup.ing.unimore.it (dbgroup.ing.unimore.it)... 155.185.48.139
Connecting to dbgroup.ing.unimore.it (dbgroup.ing.unimore.it)|155.185.48.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1672160143 (1.6G) [application/zip]
Saving to: 'data.zip'

data.zip            100%[===================>]   1.56G  29.8MB/s    in 55s     

2022-01-29 09:07:42 (29.2 MB/s) - 'data.zip' saved [1672160143/1672160143]

Archive:  /content/data.zip
  inflating: invoices.csv            


In [7]:
correctTypes= {
    'F1_kWh': float,
    'F2_kWh': float,
    'F3_kWh': float,
    'average_gas_bill_cost': float,
    'average_light_bill_cost': float,
    'average_unit_gas_cost': float,
    'average_unit_light_cost': float,
    'bill_id': int ,
    'bill_type': str,
    'billing_frequency': str,
    'date': 'datetime',
    'emission_date': 'datetime',
    'extra_fees': float,
    'gas_amount': float,
    'gas_average_cost': float,
    'gas_consumption': float,
    'gas_end_date': 'datetime',
    'gas_material_cost': float,
    'gas_offer': float,
    'gas_start_date': 'datetime',
    'gas_system_charges': float,
    'gas_transport_cost': float,
    'howmuch_pay': float,
    'light_amount': float,
    'light_average_cost': float,
    'light_consumption': float,
    'light_end_date': 'datetime',
    'light_material_cost': float,
    'light_offer': str,
    'light_offer_type': str,
    'light_start_date': 'datetime',
    'light_system_charges': float,
    'light_transport_cost': float,
    'supply_type': str,
    'total_amount': float,
    'tv': float,
}

In [15]:
#Decomentare la prima o la seconda riga in base al tipo di enviroment 

invoices = dd.read_csv('/content/invoices.csv', dtype={'billing_frequency': 'string', 'gas_offer': 'float64', 'city':'string'}, low_memory=False) #online
#invoices = dd.read_csv('invoices.csv', dtype={'billing_frequency': 'string', 'gas_offer': 'float64', 'city':'string'}, low_memory=False) #locale
utilities = invoices[['user_code', 'customer_code', 'city', 'address']]
customers = invoices[['user_code', 'nominative', 'sex', 'age']]
invoices = invoices.drop(labels=['user_code', 'customer_code', 'city', 'address', 'nominative', 'sex', 'age'], axis=1)
myBase = BaseDfBench(invoices)
myBase.df
#(14,15,29,30,31,32,33,36) mixed type

,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=80,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,object,object,object,object,object,object,float64,object,object,object,object,object,float64,float64,float64,object,uint64,object,object,object,object,object,object,object,string,bool,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
invoices.columns

Index(['bill_id', 'F1_kWh', 'F2_kWh', 'F3_kWh', 'date', 'light_start_date',
       'light_end_date', 'tv', 'gas_amount', 'gas_average_cost',
       'light_average_cost', 'emission_date', 'supply_type', 'gas_start_date',
       'gas_end_date', 'extra_fees', 'gas_consumption', 'light_consumption',
       'gas_offer', 'light_offer_type', 'light_offer', 'howmuch_pay',
       'total_amount', 'light_amount', 'average_unit_light_cost',
       'average_light_bill_cost', 'average_unit_gas_cost',
       'average_gas_bill_cost', 'billing_frequency', 'bill_type',
       'gas_system_charges', 'light_system_charges', 'gas_material_cost',
       'light_transport_cost', 'gas_transport_cost', 'light_material_cost'],
      dtype='object')

In [25]:
mismatchedTypes = myBase.find_mismatched_dtypes()
mismatchedTypesCorrect = myBase.assign_custom_types(mismatchedTypes, correctTypes)
mismatchedTypesCorrect


[{'col': 'date', 'correct_dtype': 'datetime', 'current_dtype': 'object'},
 {'col': 'light_start_date',
  'correct_dtype': 'datetime',
  'current_dtype': 'object'},
 {'col': 'light_end_date',
  'correct_dtype': 'datetime',
  'current_dtype': 'object'},
 {'col': 'tv', 'correct_dtype': float, 'current_dtype': 'object'},
 {'col': 'gas_amount', 'correct_dtype': float, 'current_dtype': 'object'},
 {'col': 'gas_average_cost',
  'correct_dtype': float,
  'current_dtype': 'object'},
 {'col': 'emission_date',
  'correct_dtype': 'datetime',
  'current_dtype': 'object'},
 {'col': 'supply_type', 'correct_dtype': str, 'current_dtype': 'object'},
 {'col': 'gas_start_date',
  'correct_dtype': 'datetime',
  'current_dtype': 'object'},
 {'col': 'gas_end_date',
  'correct_dtype': 'datetime',
  'current_dtype': 'object'},
 {'col': 'extra_fees', 'correct_dtype': float, 'current_dtype': 'object'},
 {'col': 'light_offer_type', 'correct_dtype': str, 'current_dtype': 'object'},
 {'col': 'howmuch_pay', 'correct

In [ ]:
from dask.diagnostics import ProgressBar  
missing_values = myBase.df.isna().sum()

with ProgressBar():
  percent_missing = ((missing_values / myBase.df.index.size) * 100).compute()
percent_missing

[########################################] | 100% Completed |  2min 53.6s


bill_id                     0.000000
F1_kWh                      0.001581
F2_kWh                      0.000876
F3_kWh                      0.006564
date                        0.074982
light_start_date            0.074982
light_end_date              0.074982
tv                          0.000000
gas_amount                 35.121499
gas_average_cost           40.844314
light_average_cost         56.848449
emission_date               0.000000
supply_type                 0.000000
gas_start_date              0.074982
gas_end_date                0.074982
extra_fees                  0.000000
gas_consumption            35.121490
light_consumption          54.174903
gas_offer                   0.000000
light_offer_type            0.000000
light_offer                 0.000000
howmuch_pay                 0.000000
total_amount                0.000000
light_amount               54.174779
average_unit_light_cost    56.847982
average_light_bill_cost    56.845172
average_unit_gas_cost      40.844314
a

In [ ]:
#Percentuale di valori nulli dopo il cast, da confrontare prima del cast
'''                         POST-CAST   PRE-CAST
bill_id                     0.000000   0.000000
F1_kWh                      0.001581   0.001581
F2_kWh                      0.000876   0.000876
F3_kWh                      0.006564   0.006564
date                        0.074982   0.074982
light_start_date            0.074982   0.074982
light_end_date              0.074982   0.074982
tv                          0.000000   0.000000
gas_amount                 35.121499  35.121499
gas_average_cost           40.844314  40.844314
light_average_cost         56.848449  56.848449
emission_date               0.000000   0.000000
supply_type                 0.000000   0.000000
gas_start_date              0.074982   0.074982
gas_end_date                0.074982   0.074982
extra_fees                  0.000000   0.000000
gas_consumption            35.121490  35.121490
light_consumption          54.174903  54.174903
gas_offer                   0.000000   0.000000
light_offer_type            0.000000   0.000000
light_offer                 0.000000   0.000000
howmuch_pay                 0.000000   0.000000
total_amount                0.000000   0.000000
light_amount               54.174779  54.174779
average_unit_light_cost    56.847982  56.847982
average_light_bill_cost    56.845172  56.845172
average_unit_gas_cost      40.844314  40.844314
average_gas_bill_cost      40.828062  40.828062
billing_frequency           7.476415   7.476415
bill_type                   0.000000   0.000000
gas_system_charges         35.842962  35.842962
light_system_charges       54.697178  54.697178
gas_material_cost          35.773181  35.773181
light_transport_cost       54.689300  54.689300
gas_transport_cost         35.836179  35.836179
light_material_cost        54.689366  54.689366 '''

In [26]:
myBase.cast_columns_types(mismatchedTypesCorrect)

,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=80,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,datetime64[ns],object,datetime64[ns],datetime64[ns],float64,float64,float64,float64,object,uint64,float64,float64,float64,float64,float64,float64,float64,string,bool,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
with ProgressBar():
  myBase.df = myBase.df.compute().sort_values('emission_date')

[########################################] | 100% Completed |  8min  6.3s


In [ ]:
#Questa colonna possiede solo questi 3 valori, converrebbe fare una one_hot_encoding o una mappatura dei valori 0:gas, 1:luce, 2:gas e luce (?)

myBase.df['supply_type'].drop_duplicates().compute()

0           gas
1          luce
2    gas e luce
Name: supply_type, dtype: object

In [ ]:
myBase.df.total_amount[myBase.df.total_amount == myBase.df.light_amount + myBase.df.light_amount]

In [19]:
# Costo non negativo
# fillna(0) sulle colonen dei costi
# localizzare outliers o con metodo grafico o con min e max
# sorting ?